# Project 3

Made by Giannis Taratsas(sdi1700160)

Here we are gonna import the libraries that we will use to our queries

In [1]:
import pandas as pd
import numpy as np
import csv
import os,sys
import matplotlib.pyplot as plt

from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from nltk.stem import WordNetLemmatizer

import nltk
import seaborn as sns
from nltk.corpus import stopwords as stwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize 

from sklearn import svm
from sklearn import metrics

from sklearn.naive_bayes import GaussianNB

from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import RandomForestClassifier

from numpy import interp
import string
from collections import Counter


from scipy import sparse

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import KFold,StratifiedKFold

Here we are gonna insert the csvs. The train_data set is to train our data. The test_data set has the comments that we will try to find the results. sol_test_data has the same data as the test_data set but it also includes one column with the solutuions with which we are gonna test our predictions 

In [2]:
Location=r'C:\Tede\Apal_ergasia\data\train.csv'
train_data = pd.read_csv(Location)

Location=r'C:\Tede\Apal_ergasia\data\impermium_verification_labels.csv'
sol_test_data = pd.read_csv(Location)

Location=r'C:\Tede\Apal_ergasia\data\impermium_verification_set.csv'
test_data = pd.read_csv(Location)

Here we have a function that removes all the special characters. Firstly, we delete all the words with '\\' in them (in this way we delete special characters such as \xa0. Some words are united with the \xa0 and the other special characters for example \xa0bring, We delete them too because they are not a lot and we can handle this kind of missing information. Secondly, we remove the urls and lastly we remove all the other punctuation so we have better results to our classification algorithms

In [3]:
def remove_string_special_characters(s): #function to remove special characters
    
    slash_back = r'\s*(?:[\w_]*\\(?:[\w_]*\\)*[\w_]*)'  # this is used to remove the backslashes when we find one we remove all the word
    stripped=re.sub(slash_back,"",s)
    
    p=re.compile(r'\<http.+?\>', re.DOTALL)    # this is used to remove urls 
    stripped=re.sub(p, '', stripped)
    
    stripped= re.sub('[^A-Za-z0-9]+', ' ', stripped) # in this way we remove all the other punctuation in the text
 
    # Change any white space to one space 
    stripped = re.sub('\s+', ' ', stripped) 
    
    # Remove start and end white spaces 
    stripped = stripped.strip() 
  #  if stripped != '': 
    return stripped.lower()

Now it is time to make our word vectors with CountVectorizer. The text preprocessing and cleaning is done with the function that I declared above.

In [4]:
#make bow vectorizer
bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, ngram_range = (1,1))

#make train BoW
temp = train_data.copy()
train=[]
for x in temp['Comment'].tolist():
    train.append(remove_string_special_characters(x))

bow_train = bow_vectorizer.fit_transform(train)

#make test BoW
temp = test_data.copy()
test=[]
for x in temp['Comment'].tolist():
    test.append(remove_string_special_characters(x))
    
bow_test = bow_vectorizer.transform(test)

Now it is time to use our Naive Bayes Classifier to fit and predict the results

In [5]:
# Naive Bayes bow
gnb = GaussianNB()
gnb.fit(bow_train.toarray(), train_data['Insult'].array);

The results(accuracy score and f1 metric) we get from the basic format of the naive bayes algorithm after only cleaning our data as asked are:

In [6]:
y_pred=gnb.predict(bow_test.toarray())
print("Accuracy score is:" ,metrics.accuracy_score(sol_test_data['Insult'].array, y_pred))
print("f1 score is:" ,metrics.f1_score(sol_test_data['Insult'].array, y_pred))

Accuracy score is: 0.5248322147651007
f1 score is: 0.5220522052205221


Now we are gonna try to make Naive Bayes better using the methods that are asked. Every method is going to used seperately and we are gonna see if this makes our results better or worse

## 1)Lemmatization


We will use this function to unite the words after the lemmatization to a sting

In [7]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

Now to make our bow_train and bow_test, we clean our data, then do lemmatization and lastly we make again the same sentences but now the words are lemmatized!

In [8]:
#make bow vectorizer
bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1)

#make train BoW
text_train=[]
temp = train_data.copy()
train=[]
for x in temp['Comment'].tolist():
    train.append(remove_string_special_characters(x))

lmtzr = WordNetLemmatizer()
train1 = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
              for s in train]
for i in train1:
    text_train.append(listToString(i))

bow_train = bow_vectorizer.fit_transform(text_train)

#make test BoW
text_test=[]
temp = test_data.copy()
test=[]
for x in temp['Comment'].tolist():
    test.append(remove_string_special_characters(x))
    

test1 = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
              for s in test]
for i in test1:
    text_test.append(listToString(i))
    
bow_test = bow_vectorizer.transform(text_test)

Now it is time to use our Naive Bayes Classifier to fit and predict the results

In [9]:
# Naive Bayes bow
gnb.fit(bow_train.toarray(), train_data['Insult'].array);

The results we are getting now are the following:

In [10]:
y_pred=gnb.predict(bow_test.toarray())
print("Accuracy score is:" ,metrics.accuracy_score(sol_test_data['Insult'].array, y_pred))
print("f1 score is:" ,metrics.f1_score(sol_test_data['Insult'].array, y_pred))

Accuracy score is: 0.5279642058165548
f1 score is: 0.5325653522374834


We can see that we have a small improvement both in the accuracy score and the f1 score

## 2)Stop Words

Firstly,we make a list with stopwords from wordcloud and nltk

In [11]:
stop_list = set(stwords.words('english')) 
stop_list.update(set(STOPWORDS))                        #wordcloud stopwords

We do exactly the same as above but now after cleaning and preprocessing the  text data and we also, delete the stopwords

In [12]:
#make bow vectorizer

bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1,ngram_range=(1,1),stop_words=stop_list)

#make train BoW
temp = train_data.copy()
train=[]
for x in temp['Comment'].tolist():
    train.append(remove_string_special_characters(x))

bow_train = bow_vectorizer.fit_transform(train)

#make test BoW
temp = test_data.copy()
test=[]
for x in temp['Comment'].tolist():
    test.append(remove_string_special_characters(x))
    
bow_test = bow_vectorizer.transform(test)

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['let'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


We use fit 

In [13]:
# Naive Bayes bow
gnb.fit(bow_train.toarray(), train_data['Insult'].array);

The results are the below:

In [14]:
y_pred=gnb.predict(bow_test.toarray())
print("Accuracy score is:" ,metrics.accuracy_score(sol_test_data['Insult'].array, y_pred))
print("f1 score is:" ,metrics.f1_score(sol_test_data['Insult'].array, y_pred))

Accuracy score is: 0.5225950782997763
f1 score is: 0.5213100044863167


We can see that after deleting the stopwords from the texts we had a slight decrease in our metrics

## 3)Bigrams

The only thing that changes in the bigrams is than now we use ngram_range=(2,2) in Countvectorizer. Bigrams basically create duos of words instead of taking every word separately.

In [15]:
#make bow vectorizer

bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, ngram_range=(2,2))

#make train BoW
temp = train_data.copy()
train=[]
for x in temp['Comment'].tolist():
    train.append(remove_string_special_characters(x))

bow_train = bow_vectorizer.fit_transform(train)

#make test BoW
temp = test_data.copy()
test=[]
for x in temp['Comment'].tolist():
    test.append(remove_string_special_characters(x))
    
bow_test = bow_vectorizer.transform(test)

We fit again

In [16]:
# Naive Bayes bow
gnb.fit(bow_train.toarray(), train_data['Insult'].array);

And the results are the below:

In [17]:
y_pred=gnb.predict(bow_test.toarray())
print("Accuracy score is:" ,metrics.accuracy_score(sol_test_data['Insult'].array, y_pred))
print("f1 score is:" ,metrics.f1_score(sol_test_data['Insult'].array, y_pred))

Accuracy score is: 0.5574944071588367
f1 score is: 0.5292717753450737


We must note that after using bigrams the accuracy score increased about 3% and the f1 metric increased about 1%

## 4)Laplace Smoothing

To use Laplace Smoothing we must make Multinomial Naive Bayes instead of gaussian with alpha=1. That means that we don't need to change nothing to the text data, but only clean them.

In [18]:
#make bow vectorizer

bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1)

#make train BoW
temp = train_data.copy()
train=[]
for x in temp['Comment'].tolist():
    train.append(remove_string_special_characters(x))

bow_train = bow_vectorizer.fit_transform(train)

#make test BoW
temp = test_data.copy()
test=[]
for x in temp['Comment'].tolist():
    test.append(remove_string_special_characters(x))
    
bow_test = bow_vectorizer.transform(test)

So after cleaning all the data the results are below

In [19]:
clf = MultinomialNB(alpha=1.0)
clf.fit(bow_train.toarray(), train_data['Insult'].array);
y_pred=clf.predict(bow_test.toarray())
print("Accuracy score is:" ,metrics.accuracy_score(sol_test_data['Insult'].array, y_pred))
print("f1 score is:" ,metrics.f1_score(sol_test_data['Insult'].array, y_pred))

Accuracy score is: 0.6711409395973155
f1 score is: 0.6193682030036252


After using all the methods which where mentioned to make our results better, we can easily notice that Laplace Smoothing made the bigger impact! The accuracy score in now **15% bigger** than the first one and the f1 metric **increased about 9%** ! 

We can also conclude that Muntinomial Gausian Bayes is a better fit for this dataset than Gaussian, because it gives us better results

## TF-IDF AND PART OF SPEECH ARRAY

To make a composite of TF-IDF and Part-Of-Speech Array we firstly need to make the tfidf array.

So the TFIDF array for the train and the test files are:

In [20]:
#make tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, ngram_range=(1,2), max_features=3000,
stop_words=stop_list)

#make train TFIDF
temp = train_data.copy()
train=[]
for x in temp['Comment'].tolist():
    train.append(remove_string_special_characters(x))

tfidf_train = tfidf_vectorizer.fit_transform(train)

#make test tfidf
temp = test_data.copy()
test=[]
for x in temp['Comment'].tolist():
    test.append(remove_string_special_characters(x))
    
tfidf_test = tfidf_vectorizer.transform(test)

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['let'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Now we need need to make Part Of Speech. So firstly we clean the text and find the tags for each word. Then for every sentence we find and save the percentage of the tags in each one. Lastly, we only keep the percentage of nouns, verds, adverbs and adjectives for every sentence and save them to an array. Then we concatenate the tfidf array with the Part of speech array and make them sparse again so we can use them in the classification.

In [21]:
temp = train_data.copy()
pos=[]
for x in temp['Comment'].tolist():
    pos.append(remove_string_special_characters(x)) #here we clean the text
    

diction=[]
for x in pos:
    tags = nltk.pos_tag(word_tokenize(x)) #here we make the tags for each word
    counts = Counter(tag for word,tag in tags) # we count each tag
    total=sum(counts.values())
    #we put in a list a dictionary for each sentence which contains the tags and the percentage in the sentence
    diction.append(dict((word, float(count)/total) for word,count in counts.items()))
    
T=[]

y=0
for x in diction: #for every dictionary in the list
    noun=0
    verb=0
    adverb=0
    adjective=0
    # We make all the subclasses of noun one and if it exists in the sentence we add the percenatges
    # We do this for verbs, adverbs and adjectives too
    if 'NN' in x:
        noun=noun+x['NN']
    if 'NNS' in x:
        noun=noun+x['NNS']
    if 'NNP' in x:
        noun=noun+x['NNP']
    if 'NNPS' in x:
        noun=noun+x['NNPS']
    if 'VB' in x:
        verb=verb+x['VB']
    if 'VBD' in x:
        verb=verb+x['VBD']
    if 'VBG' in x:
        verb=verb+x['VBG']
    if 'VBN' in x:
        verb=verb+x['VBN']
    if 'VBP' in x:
        verb=verb+x['VBP']
    if 'VBZ' in x:
        verb=verb+x['VBZ']
    if 'RB' in x:
        adverb=adverb+x['RB']
    if 'RBR' in x:
        adverb=adverb+x['RBR']
    if 'RBS' in x:
        adverb=adverb+x['RBS']
    if 'JJ' in x:
        adjective=adjective+x['JJ']
    if 'JJR' in x:
        adjective=adjective+x['JJR']
    if 'JJS' in x:
        adjective=adjective+x['JJS']

    T.append([noun,verb,adverb,adjective])    
#here we make the list a numoy array
T=np.array(T)
#To make the final array we concatenate tfidf array and the one that we made
final_array=np.concatenate((tfidf_train.toarray(),T),axis=1)
#And finally after creating the array we do them again sparse
final_array = sparse.csr_matrix(final_array) 

We do exactly the same for the test_data set.

In [22]:
#Same method as above(train_data set)
temp = test_data.copy()
pos=[]
for x in temp['Comment'].tolist():
    pos.append(remove_string_special_characters(x))
    

diction1=[]
for x in pos:
    tags = nltk.pos_tag(word_tokenize(x))
    counts = Counter(tag for word,tag in tags)
    total=sum(counts.values())
    diction1.append(dict((word, float(count)/total) for word,count in counts.items()))
    
T1=[]

y=0
for x in diction1:
    noun=0
    verb=0
    adverb=0
    adjective=0
    if 'NN' in x:
        noun=noun+x['NN']
    if 'NNS' in x:
        noun=noun+x['NNS']
    if 'NNP' in x:
        noun=noun+x['NNP']
    if 'NNPS' in x:
        noun=noun+x['NNPS']
    if 'VB' in x:
        verb=verb+x['VB']
    if 'VBD' in x:
        verb=verb+x['VBD']
    if 'VBG' in x:
        verb=verb+x['VBG']
    if 'VBN' in x:
        verb=verb+x['VBN']
    if 'VBP' in x:
        verb=verb+x['VBP']
    if 'VBZ' in x:
        verb=verb+x['VBZ']
    if 'RB' in x:
        adverb=adverb+x['RB']
    if 'RBR' in x:
        adverb=adverb+x['RBR']
    if 'RBS' in x:
        adverb=adverb+x['RBS']
    if 'JJ' in x:
        adjective=adjective+x['JJ']
    if 'JJR' in x:
        adjective=adjective+x['JJR']
    if 'JJS' in x:
        adjective=adjective+x['JJS']
    T1.append([noun,verb,adverb,adjective])    
    
        
T1=np.array(T1)

final_test_array=np.concatenate((tfidf_test.toarray(),T1),axis=1)

final_test_array = sparse.csr_matrix(final_test_array) 

Now we are ready to use SVM and Random Decision Forest classifiers to see the results

Firstly we try out the SVM classifier

In [23]:
#Support Vector Machines with tfidf-sof
clf = svm.SVC()
clf.fit(final_array, train_data['Insult'].array)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

The results are the below:

In [24]:
y_pred=clf.predict(final_test_array.toarray())
print("Accuracy score is:" ,metrics.accuracy_score(sol_test_data['Insult'].array, y_pred))
print("f1 score is:" ,metrics.f1_score(sol_test_data['Insult'].array, y_pred))


Accuracy score is: 0.6536912751677852
f1 score is: 0.4812332439678284


Now we are gonna test the Random Forest Classifier

In [25]:
#Random Forest

rf = RandomForestClassifier()
rf.fit(final_array, train_data['Insult'].array);

The results are below:

In [26]:
y_pred=rf.predict(final_test_array.toarray())
print("Accuracy score is:" ,metrics.accuracy_score(sol_test_data['Insult'].array, y_pred))
print("f1 score is:" ,metrics.f1_score(sol_test_data['Insult'].array, y_pred))


Accuracy score is: 0.6644295302013423
f1 score is: 0.5241116751269035


Based on the results we can see that the way i have cleaned the data and the methods that i have used give us better results on the Random Forest Classifier after comparing the metrics. Acc score is about 1% better and f1 score is 4% better with the Random Forest Algorithm

# Results Improvement

Here we are gonna to try improve the results of one classifier! This one is GridSearch!

To increase the results we will use GridSearch! GridSearch uses cross validation to find the best hyperparameters to use to train the classifier 

In [27]:
#Support Vector Machines with tfidf
clf = svm.SVC()
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['linear']}  
grid = GridSearchCV(clf, param_grid, refit = True, verbose = 3,cv=StratifiedKFold(n_splits=5),n_jobs=1)
grid.fit(final_array, train_data['Insult'].array)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.743, total=   1.3s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.744, total=   1.1s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.2s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.744, total=   1.1s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.748, total=   1.1s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.752, total=   1.1s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.743, total=   1.1s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.744, total=   1.0s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.744, total=   1.1s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.748, total=   1.1s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] .

[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.784, total=   1.4s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.773, total=   1.3s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.786, total=   1.4s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.796, total=   1.4s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.767, total=   1.4s
[CV] C=10, gamma=0.001, kernel=linear ................................
[CV] .... C=10, gamma=0.001, kernel=linear, score=0.784, total=   1.5s
[CV] C=10, gamma=0.001, kernel=linear ................................
[CV] .... C=10, gamma=0.001, kernel=linear, score=0.773, total=   1.4s
[CV] C=10, gamma=0.001, kernel=linear ................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.4min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=1,
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

The results after the GridSearch arethe below:

In [28]:
y_pred=grid.predict(final_test_array.toarray())
print("Accuracy score is:" ,metrics.accuracy_score(sol_test_data['Insult'].array, y_pred))
print("f1 score is:" ,metrics.f1_score(sol_test_data['Insult'].array, y_pred))


Accuracy score is: 0.6774049217002237
f1 score is: 0.5530068195908245


As we can see the results are better now! Accuracy is **bigger for about 2%** and f1 metric **has increased 7%** 